In [68]:
%config InteractiveShell.ast_node_interactivity="none"

Remember the Sudoku solver we covered in lecture yesterday? Let's go through it in more detail today.

# Sudoku solver with recursion ("brute force search")

*Stolen from Jelani's 2024 JamCoders lecture.

In [36]:
# is it valid to set table[a][b] to v?
#a is row, b is column, v is number. Table is a list of lists. 
def valid(a, b, v, table):
    
    # clash in the same row?
    for i in range(9):
        if table[a][i] == v:
            return False
    
    # clash in the same column?
    for i in range(9):
        if table[i][b] == v:
            return False
    
    # clash in the same 3 x 3 square?
    s = a//3
    t = b//3
    for i in range(3):
        for j in range(3):
            if table[3*s+i][3*t+j] == v:
                return False
    
    return True

In [47]:
def sudoku_solve(table):
    # find an empty cell to try to fill
    for i in range(9): #iterate over every row and every column 
        for j in range(9):
            if table[i][j] == 0: #if cell is empty 
                for v in range(1, 10): #try out all numbers from 1-9
                    if valid(i, j, v, table): #if there's no clash for a specific number
                        table[i][j] = v #put that number in the cell
                        if sudoku_solve(table): #solve the new table with that new number
                            return True #-->if its solved with that number return true
                        table[i][j] = 0 #-->if we its not solved 
                        #remove that number from the cell
                return False #if we're tried every number from 1-9 
                #without returning true then the possle isn't solved
    return True

In [1]:
#I want to see which level of recursion we are in, so I added a variable called rec_depth to track that. 
#When the function sudoku_solve is called for the first time, rec_depth is 1, when sudoku_solve
#calls itself again, rec_depth is 2, whe it calls itself again, rec_depth is 3 etc. At each point in the 
#recursive calls, I also print the current table to see what the table looks like.

def sudoku_solve(table,rec_depth):
    print('Recursion Depth....',rec_depth)
    print('===============================')
    print(pretty_print(table))
    # find an empty cell to try to fill
    for i in range(9): #iterate over every row and every column 
        for j in range(9):
            if table[i][j] == 0: #if cell is empty 
                for v in range(1, 10): #try out all numbers from 1-9
                    if valid(i, j, v, table): #if there's no clash for a specific number
                        table[i][j] = v #put that number in the cell
                        rec_depth+=1
                        if sudoku_solve(table,rec_depth): 
                            return True
                        table[i][j] = 0 #-->if we its not solved 
                        #remove that number from the cell
                return False #if we're tried every number from 1-9 
                #without returning true then the possle isn't solved
    return True

This code is implementing the backtracking algorithm discussed here: https://en.wikipedia.org/wiki/Sudoku_solving_algorithms. You can see the animation that shows you how it works:https://en.wikipedia.org/wiki/Sudoku_solving_algorithms#/media/File:Sudoku_solved_by_bactracking.gif

In [2]:
table = [
    [0,0,0,0,0,0,0,2,8],
    [0,6,0,0,0,0,0,0,7],
    [0,0,0,4,0,1,0,0,0],
    [5,0,0,9,7,0,3,0,0],
    [2,0,4,0,0,8,0,0,0],
    [3,0,0,0,0,4,5,0,0],
    [1,3,0,0,9,0,0,0,0],
    [0,5,7,0,0,0,0,9,0],
    [0,0,8,3,1,7,0,0,0]
]

In [3]:
def list_to_str(L):
    ans = ''
    for x in L:
        ans += str(x)
    return ans

def pretty_print(table):
    for i in range(3):
        for r in range(3):
            s = list_to_str(table[i*3+r][:3]) + ' ' + list_to_str(table[i*3+r][3:6]) + ' ' + list_to_str(table[i*3+r][6:])
            print(s)
        print('')

In [5]:
pretty_print(table) #Input Table

000 000 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000



In [72]:
sudoku_solve(table,1) 
pretty_print(table) #Solved table

#When we first call the function (rec_depth=1) we see what the input table looks like. 
#The first recursive call (rec_depth=2) has an input table with the a 4 placed in the first row and column because 1 has a column clash, 
#2 has a row clash and 3 has a colun clash. 

#In the 2nd recursive call (rec_depth=3) has an input table with an additional 1 in the first row and 2nd column. 
#In the 3rd recursive call (rec_depth=4) has an input table with an additional 3 in the 1st row and 3rd column. 
#In the 4th recursive call (rec_depth=5) has an input table with an additional 5 in the first row and 4th column. 
#In the 5th recursive call (rec_depth=6) has an input table with an additional 6 in the first row and 5th column. 
#In the 6th recursive call (rec_depth=7) has an input table with an additional 9 in the first row and 6th column. 
    #At this point all the columns in row 1 are filled except for the 7th column. And all numbers 1 to 9
    #result in clashes. Which means that we cannot solve the puzzle with this current configuration. So 
    #this recursive call of sudoku_solve returns False to the prior recursive call (rec_depth=6)
#Back in the 5th recursive call (rec_depth=6), all numbers from 1 to 9 were tried already and didn't work. 
    #That function sets the 1st row & 6th column back to 0 from 9. 
    #It returns False to the prior recursive call (rec_depth=5) because it has tried all numbers until 9.
#Now we're back in the 4th recursive call (rec_depth=5), which had set the 1st row and 5th column to 6. It sets it
    #back to 0. Since setting the 1st row & 5th column with 6 didn't work and 7,8,9 aren't valid, it sets that
    #cell back to 0 and returns False to the 3rd recursive call which had set the 1st row & 4th column with a 5. 
#Now we're back in the 3rd recursive call (rec_depth=4) which had set the 1st row and 4th column to a 5. Since the puzzle 
    #can't be solved with that configuration, it tries sudoku_solve with the next value, 6. 
    
#It goes on like this and at rec_depth=10 the first row seems to work for now. 
#But as we progress through every row ane column, we might find that the first row will still have to be changed
#because it may not work with any configuration of the rest of the rows. In fact the cofiguration of the first row
#is different in the solved puzzle.

Recursion Depth.... 1
000 000 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 2
400 000 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 3
410 000 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 4
413 000 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 5
413 500 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 6
413 560 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 7
413 569 028
060 000 007
000 401 000

500 970 300
204 008 000
300 004 500

130 090 000
057 000 090
008 317 000

None
Recursion Depth.... 6
413 600 028
